In [23]:
import pandas as pd
import numpy as np
import datetime


columns=['flight_id', 'origin', 'destination', 'duration', 'aircraft', 'fuel_expence', 'tickets_total_cost', 'income', 'departure_date', 'price_per_km']
data = pd.read_csv('flights.csv', names=columns)

data.head()

,flight_id,origin,destination,duration,aircraft,fuel_expence,tickets_total_cost,income,departure_date,price_per_km
0,136546,Anapa,Novokuznetsk,302,Boeing 737-300,4249,NaN,NaN,2017-01-03,NaN
1,136533,Anapa,Novokuznetsk,303,Boeing 737-300,6371,NaN,NaN,2016-12-13,NaN
2,136518,Anapa,Novokuznetsk,304,Boeing 737-300,8493,NaN,NaN,2016-12-20,NaN
3,136544,Anapa,Novokuznetsk,304,Boeing 737-300,8493,NaN,NaN,2017-01-31,NaN
4,136534,Anapa,Novokuznetsk,304,Boeing 737-300,8493,NaN,NaN,2016-12-06,NaN


In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   flight_id           193 non-null    int64  
 1   origin              193 non-null    object 
 2   destination         193 non-null    object 
 3   duration            193 non-null    int64  
 4   aircraft            193 non-null    object 
 5   fuel_expence        193 non-null    int64  
 6   tickets_total_cost  180 non-null    float64
 7   income              180 non-null    float64
 8   departure_date      193 non-null    object 
 9   price_per_km        180 non-null    float64
dtypes: float64(3), int64(3), object(4)
memory usage: 15.2+ KB


In [25]:
data.dropna(axis=0, subset=['income'], inplace=True)

In [26]:
data.describe()

,flight_id,duration,fuel_expence,tickets_total_cost,income,price_per_km
count,180.000000,180.000000,180.000000,1.800000e+02,1.800000e+02,180.000000
mean,136540.794444,74.988889,79990.483333,1.179647e+06,1.099657e+06,1146.113304
std,258.233139,25.061709,5112.594913,4.811836e+05,4.765546e+05,171.577073
min,136119.000000,49.000000,73651.000000,5.310000e+05,4.558460e+05,723.725006
25%,136309.000000,50.000000,75154.000000,7.385250e+05,6.633710e+05,1053.015401
50%,136531.500000,74.500000,78647.000000,1.068850e+06,9.895840e+05,1095.712104
75%,136775.000000,100.000000,84881.000000,1.631950e+06,1.548660e+06,1269.526260
max,136961.000000,103.000000,91247.000000,1.886000e+06,1.805363e+06,1479.960093


In [35]:
data.sample(12)

,flight_id,origin,destination,duration,aircraft,fuel_expence,tickets_total_cost,income,departure_date,price_per_km,day_of_week
39,136631,Anapa,Belgorod,50,Sukhoi Superjet-100,75154,721200.0,646046.0,2016-12-22,1025.696607,4
165,136306,Anapa,Moscow,100,Boeing 737-300,84881,1443200.0,1358319.0,2016-12-04,1113.492363,7
146,136215,Anapa,Moscow,100,Boeing 737-300,84881,1664000.0,1579119.0,2017-02-28,1294.494848,2
140,136391,Anapa,Moscow,100,Boeing 737-300,84881,1556600.0,1471719.0,2016-12-03,1206.452878,6
184,136268,Anapa,Moscow,101,Boeing 737-300,87003,1676200.0,1589197.0,2017-02-16,1302.756365,4
65,136778,Anapa,Belgorod,50,Sukhoi Superjet-100,75154,759000.0,683846.0,2017-02-22,1085.709890,3
178,136207,Anapa,Moscow,101,Boeing 737-300,87003,1567600.0,1480597.0,2016-12-27,1213.730686,2
152,136187,Anapa,Moscow,100,Boeing 737-300,84881,1873800.0,1788919.0,2016-12-01,1466.479998,4
174,136282,Anapa,Moscow,101,Boeing 737-300,87003,1518800.0,1431797.0,2017-01-16,1173.726514,1
160,136266,Anapa,Moscow,100,Boeing 737-300,84881,1566400.0,1481519.0,2017-02-27,1214.486503,1


In [28]:
data.sort_values(by='income').head(10)

,flight_id,origin,destination,duration,aircraft,fuel_expence,tickets_total_cost,income,departure_date,price_per_km
71,136807,Anapa,Belgorod,50,Sukhoi Superjet-100,75154,531000.0,455846.0,2017-02-23,723.725006
14,136642,Anapa,Belgorod,49,Sukhoi Superjet-100,73651,531000.0,457349.0,2017-01-30,726.111381
74,136844,Anapa,Belgorod,50,Sukhoi Superjet-100,75154,575100.0,499946.0,2017-02-28,793.740503
26,136887,Anapa,Belgorod,49,Sukhoi Superjet-100,73651,595200.0,521549.0,2017-01-20,828.038703
55,136922,Anapa,Belgorod,50,Sukhoi Superjet-100,75154,607800.0,532646.0,2017-02-11,845.656756
13,136620,Anapa,Belgorod,49,Sukhoi Superjet-100,73651,613500.0,539849.0,2017-02-17,857.092753
99,136823,Anapa,Belgorod,51,Sukhoi Superjet-100,76657,620400.0,543743.0,2017-02-05,863.274810
61,136937,Anapa,Belgorod,50,Sukhoi Superjet-100,75154,626100.0,550946.0,2017-01-03,874.710806
40,136888,Anapa,Belgorod,50,Sukhoi Superjet-100,75154,626100.0,550946.0,2017-02-14,874.710806
46,136678,Anapa,Belgorod,50,Sukhoi Superjet-100,75154,631200.0,556046.0,2017-01-28,882.807837


In [33]:
data['day_of_week'] = data.departure_date.apply(lambda x: datetime.date.fromisoformat(x).isoweekday())

In [34]:
data.sort_values(by='income').head(10)

,flight_id,origin,destination,duration,aircraft,fuel_expence,tickets_total_cost,income,departure_date,price_per_km,day_of_week
71,136807,Anapa,Belgorod,50,Sukhoi Superjet-100,75154,531000.0,455846.0,2017-02-23,723.725006,4
14,136642,Anapa,Belgorod,49,Sukhoi Superjet-100,73651,531000.0,457349.0,2017-01-30,726.111381,1
74,136844,Anapa,Belgorod,50,Sukhoi Superjet-100,75154,575100.0,499946.0,2017-02-28,793.740503,2
26,136887,Anapa,Belgorod,49,Sukhoi Superjet-100,73651,595200.0,521549.0,2017-01-20,828.038703,5
55,136922,Anapa,Belgorod,50,Sukhoi Superjet-100,75154,607800.0,532646.0,2017-02-11,845.656756,6
13,136620,Anapa,Belgorod,49,Sukhoi Superjet-100,73651,613500.0,539849.0,2017-02-17,857.092753,5
99,136823,Anapa,Belgorod,51,Sukhoi Superjet-100,76657,620400.0,543743.0,2017-02-05,863.274810,7
61,136937,Anapa,Belgorod,50,Sukhoi Superjet-100,75154,626100.0,550946.0,2017-01-03,874.710806,2
40,136888,Anapa,Belgorod,50,Sukhoi Superjet-100,75154,626100.0,550946.0,2017-02-14,874.710806,2
46,136678,Anapa,Belgorod,50,Sukhoi Superjet-100,75154,631200.0,556046.0,2017-01-28,882.807837,6
